# Fonctions utiles

### Import

In [16]:
import pandas as pd
import numpy as np
import os
import glob

### Conversion mathématique

In [34]:
def kN_t(kN=0, t_kg=0, unite_sortie='t'):
    '''
    Convertisseur kN <-> t, kg
        param: kN - kiloNewton
               t_kg - tonne ou kilogramme
               unite_sortie - Unité attendue en sortie de calcul : kN, t ou kg
        return: (float) Résultat du calcul de convertion
    '''
    
    if unite_sortie == 'kN':
        return t_kg/0.102
    elif unite_sortie == 't':
        return kN*0.102
    elif unite_sortie == 'kg':
        return kN*102
    else:
        print("L'unité n'est pas reconnue.\nChoix possible : \n\t- kN (kiloNewtons)\n\t- t (tonnes)\n\t- kg (kilogrammes)")
        return 0

In [35]:
def ms_kmh(ms=0, kmh=0, unite_sortie='kmh'):
    '''
    Convertisseur m/s <-> km/h
        param: ms - Mètre / seconde
               kmh - Kilomètre / heure
               unite_sortie - Unité attendue en sortie de calcul : ms ou kmh
        return: (float) Résultat du calcul de convertion
    '''
    
    if unite_sortie == 'kmh':
        return ms*3.6
    elif unite_sortie == 'ms':
        return kmh/3.6
    else:
        print("L'unité n'est pas reconnue.\nChoix possible : \n\t- kmh (km/h)\n\t- ms (m/s)")
        return 0

In [36]:
def hex_to_dec(element):
        new_list = []
        for elt in element:
            new_list.append(int(elt, 16))
            
        return new_list

### Découpage Series et conversion DataFrame

In [2]:
'''
Découpage des 'Warning Flags' et conversion en int
    param: WF_series - Warning Flags Series
    return: (DataFrame - int64) 8 colonnes contenant les warning flags par groupe de bits
'''
def warningFlags_dummies(WF_series):
    def normalisation_warningFlags(element):
        limit = 8 # Fixed by norm .nswd files
        len_elt = len(element)
        add_list = []
        for i in range(limit):
            add_list.append('0')

        if len_elt <= limit:
            alpha = limit-len_elt
            element = add_list[:alpha] + element

        return element
    
    normalisationWarningFlags = WF_series.apply(lambda x: normalisation_warningFlags(hex_to_dec(list(x))))
    warnings = pd.DataFrame.from_items(zip(normalisationWarningFlags.index, normalisationWarningFlags.values),\
                                       columns=['W7', 'W6', 'W5', 'W4', 'W3', 'W2', 'W1', 'W0'], orient='index')
    
    return warnings.astype('int64')

In [40]:
'''
Découpage du 'timestamp' et conversion en int
    param: TS_series - Timestamp Series
    return: (Dataframe - int64) 7 colonnes contenant respectivement une donnée du timestamp: Year, Month, Day, ...
'''
def timestamp_dummies(TS_series):
    splitTimestamp = TS_series.str.split('-')
    time = pd.DataFrame.from_items(zip(splitTimestamp.index, splitTimestamp.values),\
                                   columns=['Year', 'Month', 'Day', 'Hour', 'Minute', 'Seconde', 'Millisecondes'],\
                                   orient='index')
    
    return time.astype('int64')

In [38]:
'''
Découpage de 'axle groups' et conversion en int
    param: AG_series - Axle Groups Series
    return: (Dataframe - int64) x colonnes dépendant du nombre d'axes totals de AG_series contenant respectivement un
            roupe d'axe chacune
'''
def axleGroups_dummies(AG_series):
    limit = np.max(AG_series.apply(lambda x: len(x)))
    
    def normalisation_axleGroups(element, limit):
        len_elt = len(element)
        add_list = []
        for i in range(limit):
            add_list.append('0')

        if len_elt <= limit:
            alpha = limit-len_elt
            element += add_list[:alpha]

        return element
    
    normalisationAxleGroups = AG_series.apply(lambda x: normalisation_axleGroups(hex_to_dec(list(x)), limit))

    # Création colonnes axle groups
    columns = []
    base_name = 'AG_'
    for i in range(1,limit+1):
        columns.append(base_name + str(i))
    
    axle = pd.DataFrame.from_items(zip(normalisationAxleGroups.index, normalisationAxleGroups.values),\
                                       columns=columns, orient='index')
    
    return axle.astype('int64')

### Recherche sur DataFrame ou Serie

In [5]:
'''
Recherche éléments uniques dans pandas Series
    param: Series - Pandas Series où rechercher les éléments uniques
    return: (DataFrame) Ensemble des éléments uniques du pandas Series
'''
def unique_element(Series):
    element = []
    for val in Series:
        if val not in element:
            element.append(val)
            
    return pd.DataFrame(element, columns=[Series.name])

In [6]:
'''
Recherche éléments uniques dans pandas dataframe
    param: DataFrame - Pandas DataFrame où rechercher les éléments uniques
    return: (DataFrame) Ensemble des éléments uniques de la pandas DataFrame
'''
def unique_element_df(DataFrame):
    df = pd.DataFrame()
    for col in DataFrame:
        df = pd.concat([df, unique_element(DataFrame[col])], axis=1)

    return df

In [1]:
def count_null(DataFrame):
    '''
    Comptage des élements NaN par colonne dans DataFrame
        param: DataFrame - DataFrame où l'on cherche à compter les NaN 
        return: (DataFrame) Listant l'ensemble des colonnes en comptant leur nombre de NaN
    '''
    return pd.DataFrame(data={'Missing':DataFrame.isnull().sum()}).sort_values(['Missing'])

### Liste redondante

In [8]:
'''
Création liste charge sur essieux et distance entre essieux
    param: bare_str_var - String contenant le préfix de la variable avant le chiffre incrémenté
           var_from - Int de départ de l'incrémentation de la liste
           var_to - Int de fin de la liste
    return: (list) Liste de l'ensemble des variables incrémentés commençant par 'bare_str_var'
'''
def list_multiple_var(bare_str_var, var_from=1, var_to=1):
    list_return = []
    
    try:
        if var_from > var_to:
            raise ValueError
        
        for i in range(var_from, var_to+1):
            list_return.append(bare_str_var+'{}'.format(i))
            
        return list_return
        
    except ValueError:
        print("'var_from' doit être inférieur ou égale à 'var_to'")

### Fichiers système

In [2]:
def find_all_extension(source, extension):
    '''
        Récupérer chemins des fichiers avec 'extension'
        param:
            - (source : str) String de chemin parent ou commencer à chercher les fichiers
            - (extension : str) String de l'extension à rechercher
        return: (list) Tous les chemins des fichiers avec 'extension'
    '''
    
    list_ext = []
    for elt in glob.glob(source):
        if os.path.isdir(elt):
            list_ext.extend(find_all_extension(elt + '\\*', extension))
        else:
            if extension in elt:
                list_ext.append(elt)
          
    return list_ext

In [1]:
def nswd_association(path, dest_file):
    '''
        Association en un fichier des .nswd d'un dossier
        param:
            - (path : str) String de chemin où se situe les fichiers .nswd
            - (dest_file : str) String du nom de fichier qui contiendra tous l'association des fichiers
        return: /
    '''    
    
    dest_nswd = path + '\\' + dest_file
    extension = '.nswd'

    nswd = find_all_extension(path, extension)
    with open(dest_nswd, 'w') as file_dest:
        for elt in nswd:
            with open(elt, 'r') as file_src:
                result = file_src.read()
                file_dest.write(result)

In [ ]:
def save_sig_trucks_json(dict_trucks, directory="backup/", filename="backup_trucks.json"):
    '''
        Conversion et sauvegarde des meilleurs signaux selectionnés pour chacun des camions dans un fichier .json.
        __________
        param:
            - (filename : str) Nom de fichier de destination de la sauvegarde
            - (directory : str) Dossier de sauvegarde des fichiers .json
            - (dict_trucks : dict) Dictionnaire (key : nom fichier, value: dataframe time, amplitude)
    '''
    
    trucks_save = {}
    for name_truck, signaux_truck in dict_trucks.items():
        # Creation dict pour sauvegarde
        trucks_save[name_truck] = signaux_truck.sort_index().to_json()
    
    if not os.path.exists(directory):
        os.mkdir(directory)
    
    with open(directory+filename, 'w') as file:
        json.dump(trucks_save, file)
        

In [ ]:
def json_sig_trucks_load(directory="backup/", filename="backup_trucks.json"):
    '''
        Chargement et conversion des meilleurs signaux selectionnés pour chacun des camions dans un dictionnaire (key : nom fichier, value: dataframe time, amplitude).
        __________
        param:
            - (directory : str) Dossier de sauvegarde des fichiers .json
            - (filename : str) Nom de fichier de destination de la sauvegarde
        __________
        return: Dictionnaire chargé du json
    '''
    
    with open(directory+filename, 'r') as file:
        trucks_saved = json.load(file)
        
    trucks = {}
    for name_truck, signaux_truck in trucks_saved.items():
        trucks[name_truck] = pd.read_json(signaux_truck).sort_index().T.reindex(['time', 'amplitude']).T
        
    return trucks

### Signaux event

In [1]:
def recover_sig(file, sig_list):
    '''
        Récupération du ou des signaux des poids lourds voulu
        __________
        
        param:
            - (file : str) Chemin du fichier où récupérer les signaux
            - (sig_list : list) Liste des signaux à récupérer
        __________
            
        return: (DafaFrame) contenant le temps et une colonne par signal pour le fichier passé
    '''
    
    # Récupération de tous les signaux de 'file'
    data = []
    with open(file, 'r') as f:
        for line in f:
            data.append(line.split())
    
    # Index des signaux voulu
    numbers = []
    for sig in sig_list:
        numbers.append(int(sig[-2:]))
    
    sig_df = pd.DataFrame(data).iloc[:,[0] + numbers]
    
    # Création nom colonne
    sig_df.columns = ['time'] + sig_list
    
    # Création nom signal
    name = file.split('\\')[-1].split('.')[0]
    
    return (name, sig_df.astype('float64'))

In [ ]:
def select_best_signal(signaux_truck):
    '''
        Calcul rapport Signal à bruit pour connaître le meilleur signal et le récupérer
        __________
        
        param:
            - (signaux_truck : DataFrame) Ensemble des signaux récupérer sur tous les capteurs d'un camion
        __________
        
        return: (DataFrame) contenant le meilleur signal avec time et amplitude
    '''
    
    # Calcul rapport signal à bruit (Plus c'est grand, mieux c'est !)
    S_N = pd.DataFrame()
    for signal in signaux_truck.columns[1:]:
        df = pd.DataFrame([[signal, signaux_truck.loc[:,signal].std()/signaux_truck.loc[:int(0.3*len(signaux_truck)),signal].std()]],\
                          columns=["Name", "S/N"])
        S_N = S_N.append(df)
        
    # Selection meilleur signal
    S_N_sort = S_N.reset_index(drop=True).sort_values(by="S/N", ascending=False)
    best_signal = pd.DataFrame([signaux_truck.loc[:,signaux_truck.columns[0]],\
                                signaux_truck.loc[:,S_N_sort.loc[0,"Name"]]]).T
    best_signal.columns = ["time", "amplitude"]
    
    return best_signal

In [ ]:
def select_all_best_signal(search_path, extension='.txt', backup_path="backup/backup_trucks.json"):
    '''
        Récupération du meilleur signal pour chacun des camions du 'search_path' donné
        __________
        
        param:
            - (search_path : str) Chemin dans lequel rechercher les fichiers avec 'extension'
            - (extension : str) Extension à rechercher
            - (backup_path : str) Chemin d'enregistrement de la sauvegarde
        __________
        
        return: (Dictionnaire) avec key : nom fichier et value: dataframe time, amplitude
    '''
    
    if not os.path.exists(backup_path):
        trucks = {}
        files = find_all_extension(search_path, extension)
        for file in files:
            # Récupération des signaux
            siwim_sig = list_multiple_var('A0', var_to=9) + list_multiple_var('A', 10, 21)
            signaux = recover_sig(file, siwim_sig)
            trucks[signaux[0]] = signaux[1]
    
    
        for name_truck, signaux_truck in trucks.items():
            # Calcul et selection meilleur signal
            best_signal = select_best_signal(signaux_truck)

            # Remplacement tout signaux par meilleur signal et remise à 0
            trucks[name_truck] = supperposition_signaux(best_signal, 10)
            
            # Sauvegarde meilleurs signaux
            bkp_path = backup_path.split("/")
            save_sig_trucks_json(trucks, filename=bkp_path[1], directory=bkp_path[0]+"/")
            
    else:
        # Récupération signaux sauvegardés
        bkp_path = backup_path.split("/")
        trucks = json_sig_trucks_load(directory=bkp_path[0]+"/", filename=bkp_path[1])
        
    return trucks

In [19]:
def supperposition_signaux(signaux, delta):
    '''
        Supperposition des signaux en les remontants à 0
        __________
        
        param:
            - (signaux : DataFrame) Dataframe des signaux intéressants du poids lourds
            - (delta : int) Fraction du signal à récupérer pour moyenner le points de superposition
        __________
        
        return: (DataFrame) contenant les signaux superposés
    '''
    
    signaux_superposé = signaux
    
    # Moyenne 1er dixième
    fst_mean = signaux.iloc[:int(len(signaux)/delta),:].iloc[:,1].mean()
    
    # Distance entre les signaux et 0
    d = 0 - fst_mean
    
    signaux_superposé.iloc[:,1:] = signaux.iloc[:,1:].apply(lambda x: x+d, axis=1)
    
    return signaux_superposé

### Instance de class

In [ ]:
def save_instance_class(dict_trucks_class, directory="backup/instance/", filename="class_trucks.sig"):
    '''
        Sauvegarde de toutes les instances 'Signal' créées lors de la récupération des fichiers signaux
        __________
        
        param:
            - (dict_trucks_class : dict) Dictionnaire des class contenant comme clé, le nom du fichier et comme valeur
                l'instance de class
            - (directory : str) Dossier de sauvegarde des instances de class
            - (filename : str) Nom du fichier de sauvegarde des instances de class
    '''
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    with open(directory+filename, 'wb') as file:
        cloudpickle.dump(list_trucks_class, file)

In [ ]:
def load_instance_class(directory="backup/instance/", filename="class_trucks.sig"):
    '''
        Récupération de toutes les instances 'Signal' sauvegardé dans 'directory' avec le nom 'filename'.
        __________
        
        param:
            - (directory : str) Dossier de sauvegarde des instances de class
            - (filename : str) Nom du fichier de sauvegarde des instances de class
        __________
        
        return: (Dictionnaire) contenant comme clé, le nom du fichier et comme valeur l'instance de class
    '''
    with open(directory+filename, 'rb') as file:
        trucks = cloudpickle.load(file)
        
    return trucks